In [1]:
from scanbox.nlw_controller import BoxController

box = BoxController('COM4')



In [2]:
box.set_continuous_resonant(True)


bytearray(b'4\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00')


In [3]:
box.set_linescan_mode('bidi')


bytearray(b'5\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00')


In [12]:
box.set_shutter(0)


bytearray(b'\x10\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')
